In [18]:
import yfinance as yf
from datetime import datetime, date
import pandas as pd
import ta

# Data Extraction

## Historical data on the world's main instruments (percentage variation)

In [54]:
# Descargar datos históricos del EUR/USD y USD/JPY desde 2015
def get_data():
    """
    Obtiene los datos históricos de los principales instrumentos financieros
    """
    # Definir los símbolos y período
    symbols = {
        # Pares de divisas
        "EURUSD": "EURUSD=X",  # Símbolo de Yahoo Finance para EUR/USD
        "USDJPY": "USDJPY=X",   # Símbolo de Yahoo Finance para USD/JPY
        "GBP/USD": "GBPUSD=X",
        "USD/CHF": "USDCHF=X",
        "AUD/USD": "AUDUSD=X",
        "USD/CAD": "USDCAD=X",

        # Índices bursátiles
        "S&P 500": "^GSPC",
        "Nasdaq 100": "^NDX",
        "Dow Jones": "^DJI",
        "DAX 40": "^GDAXI",
        "FTSE 100": "^FTSE",
        "Nikkei 225": "^N225",
        "Euro Stoxx 50": "^STOXX50E",
        "Hang Seng": "^HSI",

        # Commodities
        "Petróleo WTI": "CL=F",
        "Petróleo Brent": "BZ=F",
        "Oro": "GC=F",
        "Plata": "SI=F",
        "Cobre": "HG=F",
        "Gas Natural": "NG=F",

        # Acciones
        "Apple": "AAPL",
        "Microsoft": "MSFT",
        "Amazon": "AMZN",
        "Alphabet": "GOOGL"
    }
    start_date = "2015-01-01"
    end_date = date.today().strftime("%Y-%m-%d")
    
    print(f"Descargando datos desde {start_date} hasta {end_date}...")
    
    # DataFrame principal que contendrá todos los datos
    df_combined = None
    
    for pair_name, symbol in symbols.items():
        print(f"Procesando {pair_name}...")
        
        try:
            # Descargar datos
            data = yf.download(symbol, start=start_date, end=end_date, progress=False)
            
            if data.empty:
                print(f"No se pudieron obtener datos para {pair_name}. Probando símbolo alternativo...")
                # Intentar con un símbolo alternativo si el original falla
            
            if not data.empty:
                # Crear DataFrame temporal
                df_temp = pd.DataFrame()
                df_temp['Fecha'] = data.index
                df_temp['Precio_Cierre'] = data['Close'].values
                
                # Calcular variación porcentual diaria
                df_temp[f'Variacion_Porcentual {pair_name}'] = ((df_temp['Precio_Cierre'] - df_temp['Precio_Cierre'].shift(1)) / df_temp['Precio_Cierre'].shift(1)) * 100
                
                # Crear columna con la variación del día siguiente
                df_temp[f'Variacion_Porcentual_Dia_Anterior {pair_name}'] = df_temp[f'Variacion_Porcentual {pair_name}'].shift(-1)
                
                # Reemplazar los nan por cero
                df_temp[f'Variacion_Porcentual {pair_name}'] = df_temp[f'Variacion_Porcentual {pair_name}'].fillna(0)
                df_temp[f'Variacion_Porcentual_Dia_Anterior {pair_name}'] = df_temp[f'Variacion_Porcentual_Dia_Anterior {pair_name}'].fillna(0)
                
                # Seleccionar columnas relevantes
                columns_to_keep = ['Fecha', f'Variacion_Porcentual {pair_name}', f'Variacion_Porcentual_Dia_Anterior {pair_name}']
                df_temp = df_temp[columns_to_keep]
                
                # Combinar con el DataFrame principal
                if df_combined is None:
                    df_combined = df_temp.copy()
                else:
                    df_combined = pd.merge(df_combined, df_temp, on='Fecha', how='outer')
                
                print(f"Datos de {pair_name} procesados exitosamente")
            else:
                print(f"No se pudieron obtener datos para {pair_name}")
                
        except Exception as e:
            print(f"Error al descargar datos de {pair_name}: {e}")
    
    if df_combined is not None and not df_combined.empty:
        # Ordenar por fecha
        df_combined = df_combined.sort_values('Fecha').reset_index(drop=True)
        
        # Formatear la fecha
        df_combined['Fecha'] = pd.to_datetime(df_combined['Fecha']).dt.strftime('%Y-%m-%d')
        
        # Rellenar valores NaN con 0 (en caso de fechas faltantes)
        df_combined = df_combined.fillna(0)
        
        print(f"Datos combinados obtenidos exitosamente: {len(df_combined)} registros")
        print(f"Período: {df_combined['Fecha'].iloc[0]} a {df_combined['Fecha'].iloc[-1]}")
        
        return df_combined
    else:
        print("No se pudieron obtener datos de ningún par de divisas")
        return None

# Ejecutar la función
df_data = get_data()

# Descargar el DataFrame a un archivo CSV
if df_data is not None:
    df_data.to_csv('data/df_data_daily_variation.csv', index=False)
    print("Datos guardados en 'data/fdf_data_daily_variation.csv'")

# Mostrar el DataFrame
df_data

Descargando datos desde 2015-01-01 hasta 2025-06-09...
Procesando EURUSD...
Datos de EURUSD procesados exitosamente
Procesando USDJPY...
Datos de USDJPY procesados exitosamente
Procesando GBP/USD...
Datos de GBP/USD procesados exitosamente
Procesando USD/CHF...
Datos de USD/CHF procesados exitosamente
Procesando AUD/USD...
Datos de AUD/USD procesados exitosamente
Procesando USD/CAD...
Datos de USD/CAD procesados exitosamente
Procesando S&P 500...
Datos de S&P 500 procesados exitosamente
Procesando Nasdaq 100...
Datos de Nasdaq 100 procesados exitosamente
Procesando Dow Jones...
Datos de Dow Jones procesados exitosamente
Procesando DAX 40...
Datos de DAX 40 procesados exitosamente
Procesando FTSE 100...
Datos de FTSE 100 procesados exitosamente
Procesando Nikkei 225...
Datos de Nikkei 225 procesados exitosamente
Procesando Euro Stoxx 50...
Datos de Euro Stoxx 50 procesados exitosamente
Procesando Hang Seng...
Datos de Hang Seng procesados exitosamente
Procesando Petróleo WTI...
Datos de

,Fecha,Variacion_Porcentual EURUSD,Variacion_Porcentual_Dia_Anterior EURUSD,Variacion_Porcentual USDJPY,Variacion_Porcentual_Dia_Anterior USDJPY,Variacion_Porcentual GBP/USD,Variacion_Porcentual_Dia_Anterior GBP/USD,Variacion_Porcentual USD/CHF,Variacion_Porcentual_Dia_Anterior USD/CHF,Variacion_Porcentual AUD/USD,...,Variacion_Porcentual Gas Natural,Variacion_Porcentual_Dia_Anterior Gas Natural,Variacion_Porcentual Apple,Variacion_Porcentual_Dia_Anterior Apple,Variacion_Porcentual Microsoft,Variacion_Porcentual_Dia_Anterior Microsoft,Variacion_Porcentual Amazon,Variacion_Porcentual_Dia_Anterior Amazon,Variacion_Porcentual Alphabet,Variacion_Porcentual_Dia_Anterior Alphabet
0,2015-01-01,0.000000,-0.076165,0.000000,0.164620,0.000000,-0.007796,0.000000,-0.001007,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2015-01-02,-0.076165,-1.182694,0.164620,0.470506,-0.007796,-1.892271,-0.001007,1.176947,0.081778,...,0.000000,-4.029306,0.000000,-2.817151,0.000000,-0.919553,0.000000,-2.051729,0.000000,-1.905400
2,2015-01-05,-1.182694,-0.062087,0.470506,-0.837799,-1.892271,-0.173944,1.176947,0.039764,-1.347322,...,-4.029306,1.943095,-2.817151,0.009398,-0.919553,-1.467745,-2.051729,-2.283333,-1.905400,-2.467944
3,2015-01-06,-0.062087,-0.533201,-0.837799,-0.629689,-0.173944,-0.788688,0.039764,0.506859,0.396898,...,1.943095,-2.280460,0.009398,1.402207,-1.467745,1.270550,-2.283333,1.059974,-2.467944,-0.294090
4,2015-01-07,-0.533201,-0.331407,-0.629689,0.519078,-0.788688,-0.200927,0.506859,0.340158,-0.242406,...,-2.280460,1.950540,1.402207,3.842272,1.270550,2.941782,1.059974,0.683602,-0.294090,0.348405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2717,2025-06-03,0.881923,-0.602247,-0.947750,0.972264,0.664281,-0.186736,-0.769815,0.851774,0.856482,...,0.757983,-0.161198,0.778387,-0.221379,0.216464,0.194396,-0.454869,0.738899,-1.686095,1.125292
2718,2025-06-04,-0.602247,0.330095,0.972264,-0.768238,-0.186736,0.153141,0.851774,-0.597894,-0.497243,...,-0.161198,-1.049517,-0.221379,-1.079776,0.194396,0.821350,0.738899,0.328142,1.125292,0.095212
2719,2025-06-05,0.330095,0.278328,-0.768238,0.567506,0.153141,0.202333,-0.597894,0.156486,0.380355,...,-1.049517,2.909978,-1.079776,1.639831,0.821350,0.577320,0.328142,2.722333,0.095212,3.251879
2720,2025-06-06,0.278328,0.000000,0.567506,0.800453,0.202333,-0.304608,0.156486,0.245345,0.305348,...,2.909978,0.000000,1.639831,0.000000,0.577320,0.000000,2.722333,0.000000,3.251879,0.000000
